# 💬 Chipchat - 데이터시트 챗봇

**전처리된 데이터시트 JSON 파일을 기반으로 질의응답을 수행하는 챗봇입니다.**

---

## 🚀 시작하기 전에

이 노트북은 Google Colab에서 Chipchat app을 실행하는 방법을 제공합니다.


## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: Google Drive 연동 { display-mode: "form" }
#@markdown Google Drive를 연동하여 전처리된 JSON 파일과 벡터 스토어를 관리합니다.

from google.colab import drive
import os
from pathlib import Path

print("🔄 Google Drive 마운트 중...")
drive.mount('/content/drive')

# JSON 및 벡터 스토어 폴더 설정
json_folder = Path('/content/drive/MyDrive/json_key')
vectorstore_folder = Path('/content/drive/MyDrive/vectorstore')
prompt_templates_folder = Path('/content/drive/MyDrive/prompt_templates')

# 폴더가 없으면 생성
json_folder.mkdir(parents=True, exist_ok=True)
vectorstore_folder.mkdir(parents=True, exist_ok=True)
prompt_templates_folder.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Google Drive 연동 완료!")
print(f"📁 JSON 폴더: {json_folder}")
print(f"📁 벡터 스토어 폴더: {vectorstore_folder}")
print(f"📁 프롬프트 템플릿 폴더: {prompt_templates_folder}")

# Vectorestore 생성 확인
vectorstore_files = list(json_folder.glob("*.json"))
print(f"\n📄 발견된 store : {len(vectorstore_files)}개")
for json_file in vectorstore_files[:5]:  # 처음 5개만 표시
    print(f" - {json_file.name}")
    
if len(vectorstore_files) > 5:
    print(f" ... 외 {len(vectorstore_files) - 5}개")

if not vectorstore_files:
    print("\n⚠️ Vectorstore가 없습니다. PDF 전처리 모듈을 먼저 실행하여 Vectorstore를 생성해주세요.")

In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
from pathlib import Path

print("📥 필요한 라이브러리 설치 중...")

!pip install -r requirements.txt -q
!pip install pyngrok==7.0.1 -q

# 현재 디렉토리 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# 새로운 디렉토리 구조 추가
src_dir = current_dir / 'src'
if src_dir.exists():
    sys.path.append(str(src_dir))
    for subdir in ['config', 'models', 'utils', 'app']:
        subdir_path = src_dir / subdir
        if subdir_path.exists():
            sys.path.append(str(subdir_path))

print("✅ 라이브러리 설치 완료!")

In [ ]:
#@title 🔑 3단계: API 키 설정 { display-mode: "form" }
#@markdown AI 서비스의 API 키를 입력하여 챗봇을 설정합니다.

import os
import json

openai_api_key = "" #@param {type:"string"}
claude_api_key = "" #@param {type:"string"}
hf_token = "" #@param {type:"string"}

# API 키 유효성 검사
if not openai_api_key and not claude_api_key and not hf_token:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 최소 하나 이상의 API 키를 입력해주세요.")
    print("• OpenAI API 키: https://platform.openai.com/api-keys")
    print("• Claude API 키: https://console.anthropic.com/keys")
    print("• HuggingFace 토큰: https://huggingface.co/settings/tokens")
else:
    # 환경 변수 설정
    if openai_api_key:
        os.environ["OPENAI_API_KEY"] = openai_api_key
        print("✅ OpenAI API 키 설정 완료!")
        
    if claude_api_key:
        os.environ["ANTHROPIC_API_KEY"] = claude_api_key
        print("✅ Claude API 키 설정 완료!")
    
    if hf_token:
        os.environ["HF_TOKEN"] = hf_token
        print("✅ HuggingFace 토큰 설정 완료!")
    
    # Streamlit 시크릿 파일 생성
    secrets_dir = Path(".streamlit")
    secrets_dir.mkdir(exist_ok=True)
    
    secrets = {}
    if openai_api_key:
        secrets["openai_api_key"] = openai_api_key
    if claude_api_key:
        secrets["anthropic_api_key"] = claude_api_key
    if hf_token:
        secrets["hf_token"] = hf_token
    
    with open(secrets_dir / "secrets.toml", "w") as f:
        for key, value in secrets.items():
            f.write(f'{key} = "{value}"\n')
    
    # 토큰 저장
    try:
        from src.config.token_manager import TokenManager
        token_manager = TokenManager()
        
        if openai_api_key:
            token_manager.set_token('openai', openai_api_key)
        
        if claude_api_key:
            token_manager.set_token('anthropic', claude_api_key)
        
        if hf_token:
            token_manager.set_token('huggingface', hf_token)
            
        print("✅ 토큰 관리자에 API 키 저장 완료!")
    except Exception as e:
        print(f"❌ 토큰 저장 중 오류가 발생했습니다: {str(e)}")
    
    # 기본 프롬프트 템플릿 생성
    default_template = {
        "pre": "당신은 전자 부품 데이터시트에 대해 응답하는 도우미입니다. 제공된 컨텍스트 정보를 기반으로 질문에 답변하세요.",
        "post": "검색된 정보를 바탕으로 명확하고 간결하게 답변해주세요."
    }
    
    template_file = prompt_templates_folder / "default_template.json"
    if not template_file.exists():
        with open(template_file, 'w', encoding='utf-8') as f:
            json.dump(default_template, f, ensure_ascii=False, indent=2)
            
        print("✅ 기본 프롬프트 템플릿이 생성되었습니다.")

In [ ]:
#@title 📊 4단계: Streamlit 서버 실행 { display-mode: "form" }
#@markdown Streamlit 서비스를 실행하여 웹 인터페이스를 제공합니다.

import subprocess
import threading
import time
import os
import json
from pathlib import Path
from pyngrok import ngrok

# 환경 변수 설정
os.environ['VECTORSTORE_PATH'] = str(vectorstore_folder)
os.environ['JSON_FOLDER_PATH'] = str(json_folder)
os.environ['PROMPT_TEMPLATES_PATH'] = str(prompt_templates_folder)

# Streamlit 실행 함수
def run_streamlit():
    cmd = [
        "streamlit", "run", 
        "src/app/streamlit_app.py",  # 경로 변경
        "--server.port=8501", 
        "--server.address=localhost"
    ]
    process = subprocess.Popen(
        cmd, 
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    return process

# Ngrok 설정 (공개 URL 만들기)
ngrok_token = "" #@param {type:"string"}

if ngrok_token:
    ngrok.set_auth_token(ngrok_token)

# Streamlit 서버 실행
print("🚀 Streamlit 서버를 시작합니다...")
process = run_streamlit()
time.sleep(5)  # 서버 시작 대기

# Ngrok 터널 설정
try:
    public_url = ngrok.connect(8501).public_url
    print(f"\n✅ 서버가 시작되었습니다!")
    print(f"\n🔗 다음 URL을 통해 AutoDataSheet에 접속할 수 있습니다:")
    print(f"📱 {public_url}")
    print("\n이 URL은 세션이 유지되는 동안에만 유효합니다.")
    print("URL을 클릭하면 AutoDataSheet 웹 인터페이스로 이동합니다.")
except Exception as e:
    # ngrok이 실패하면 Colab의 웹 프록시 사용
    print("\n⚠️ ngrok 연결 중 오류가 발생했습니다. Colab의 웹 프록시를 사용합니다.")
    
    from google.colab.output import eval_js
    url = eval_js('google.colab.kernel.proxyPort(8501)')
    
    print(f"\n✅ 서버가 시작되었습니다!")
    print(f"\n🔗 다음 URL을 통해 AutoDataSheet에 접속할 수 있습니다:")
    print(f"📱 {url}")

In [ ]:
#@title 🛑 5단계: 서버 중지 { display-mode: "form" }
#@markdown 작업을 마치면 서버를 중지합니다.

import os
import signal
from pyngrok import ngrok

# 서버 중지 여부 확인
stop_server = True  #@param {type:"boolean"}

if stop_server:
    # ngrok 터널 종료
    try:
        ngrok.kill()
        print("✅ ngrok 터널이 종료되었습니다.")
    except:
        pass
    
    # Streamlit 프로세스 종료
    try:
        # 모든 streamlit 프로세스 종료
        !pkill -f streamlit
        print("✅ Streamlit 서버가 종료되었습니다.")
    except:
        print("⚠️ Streamlit 서버 종료 중 오류가 발생했습니다.")
        print("💡 수동으로 런타임을 재시작하여 서버를 종료할 수 있습니다.")

---

## 🛠️ 문제 해결

**애플리케이션이 실행되지 않는 경우:**

1. **API 키 확인**: OpenAI API 키가 올바르게 입력되었는지 확인
2. **세션 재시작**: 런타임 → 세션 재시작 후 처음부터 다시 실행
3. **JSON 파일 확인**: 전처리된 JSON 파일이 Google Drive에 존재하는지 확인

**사용 중 문제가 발생하는 경우:**
- 브라우저를 새로고침하세요
- 네트워크 연결 상태를 확인하세요

---

## 📞 지원

추가 도움이 필요하시면 GitHub 이슈를 등록해주세요:
🔗 https://github.com/doyoung42/chipchat/issues
